# Processing Experiment Results

In [32]:
# Imports
import time
import logging
import random
import datetime
import pprint
import traceback
pp = pprint.PrettyPrinter()

# logging
logging.basicConfig(level=logging.FATAL)
LOGGER = logging.getLogger()

# Data Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
package_names = ['YOUR_HUB_ID/cpu-automated', 'YOUR_HUB_ID/fileio-automated', 'YOUR_HUB_ID/oltp-automated']
test_names = ['binpack', 'dispatch', 'random','ml']

In [ ]:
test_markers = ['o', '^', 'v', 'x']
j=0
names = []
for test_name in test_names:
    if test_name == 'ml':
        names.append('Smart Spread')
    elif test_name == 'binpack':
        names.append('Binpack')
    elif test_name == 'dispatch':
        names.append('Spread')
    elif test_name == 'random':
        names.append('Random')
    else:
        names.append(test_name)
    
    marker = test_markers[j]
    j += 1
    
    test_res = pd.read_csv('Data/' + test_name + '.csv')
    
    sequence_len = test_res.shape[0]
    
    latencies = np.zeros((sequence_len,))
    throughputs = np.zeros((sequence_len,))
    counts = np.zeros((sequence_len,))
    tp_per_nodes = np.zeros((sequence_len,))
    
    for package_name in package_names:
        l = test_res[package_name + '-avg_ms']
        t = test_res[package_name + '-completed']
        c = test_res[package_name + '-count']
        
        user_count_sequence = test_res[package_name + '-user_count']
        
        latencies = np.add(latencies, np.array(l))
        throughputs = np.add(throughputs, np.array(t))
        counts = np.add(counts, np.array(c))
        
        tp_per_node = np.divide(t, c)
        tp_per_nodes = np.add(tp_per_nodes, tp_per_node)
        
    plt.figure(1)
    plt.plot(latencies, marker = marker)
    plt.ylabel('Response Time (ms)')
    plt.xlabel("Time in Minutes")
    plt.figure(2)
    plt.plot(throughputs, marker = marker)
    plt.ylabel('Throughput (Req/min)')
    plt.xlabel("Time in Minutes")
    plt.figure(3)
    plt.plot(counts, marker = marker)
    plt.legend(package_names)
    plt.ylabel('Container Count (#)')
    plt.xlabel("Time in Minutes")
    plt.figure(4)
    plt.plot(user_count_sequence, marker = marker)
    plt.ylabel("Concurrency Level")
    plt.xlabel("Time in Minutes")
    plt.figure(5)
    plt.plot(tp_per_nodes, marker = marker)
    plt.ylabel("TP per container")
    plt.xlabel("Time in Minutes")
    
    
    plt.figure(6)
    plt.plot(user_count_sequence, tp_per_nodes, marker = marker)
    plt.ylabel("Throughput")
    plt.xlabel("Concurrency Level")
    
    
    results_start_point = 10
    results_end_point = 12
    latencies = latencies[results_start_point:results_end_point]
    throughputs = throughputs[results_start_point:results_end_point]
    
    print("Test Names: ", test_name)
    print('Response Time: {:4.02f} ({:4.02f})'.format(np.mean(latencies), np.std(latencies)))
    print('Thoughput: {:4.02f} ({:4.02f})'.format(np.mean(throughputs), np.std(throughputs)))
    print('Total Package Count: {:4.02f} ({:4.02f})'.format(np.mean(counts), np.std(counts)))
    print('')
    
def shared_plot_stuff():
    plt.axvspan(7, 14, alpha=0.1, color='red')
    
package_selection_method = 'all'
package_short_name = 'all'
plt.figure(1)
shared_plot_stuff()

xlim,ylim = plt.xlim(), plt.ylim()
plt.plot([xlim[0], xlim[1]], [1700, 1700],
          linestyle="--")
plt.legend(names + ['Sum of Thresholds'])
# plt.legend(names)

plt.savefig("Images/ResponseTime_" + package_short_name + '_' + package_selection_method + ".png")
plt.figure(2)
shared_plot_stuff()
plt.legend(names)
plt.savefig("Images/Thoughput_" + package_short_name + '_' + package_selection_method + ".png")
plt.figure(3)
shared_plot_stuff()

xlim,ylim = plt.xlim(), plt.ylim()
plt.plot([xlim[0], xlim[1]], [24, 24],
          linestyle="--")
plt.legend(names + ['Maximum'])
# plt.legend(names)

plt.savefig("Images/ContainerCount_" + package_short_name + '_' + package_selection_method + ".png")
plt.figure(4)
shared_plot_stuff()
# plt.legend(names)
plt.savefig("Images/UserCountSequence_all.png")
plt.figure(5)
shared_plot_stuff()
plt.legend(names)
plt.savefig("Images/TP_per_container_" + package_short_name + '_' + package_selection_method + ".png")

In [ ]:
i = 0
test_markers = ['o', 'x', '^', 'v']
for package_name in package_names:
    save = True
    
    plt.figure(1+i, clear=True)
    plt.figure(2+i, clear=True)
    plt.figure(3+i, clear=True)
    plt.figure(4+i, clear=True)
    
    j = 0
    names = []
    for test_name in test_names:
        test_marker = test_markers[j]
        j += 1
        
        name = 'none'
        if package_name == 'nimamahmoudi/cpu-automated':
            name = 'SysbenchCPU'
        elif package_name == 'nimamahmoudi/fileio-automated':
            name = 'Fileio'
        elif package_name == 'nimamahmoudi/oltp-automated':
            name = 'OLTP'
        
        if test_name == 'ml':
            names.append('Smart Spread')
        elif test_name == 'binpack':
            names.append('Binpack')
        elif test_name == 'dispatch':
            names.append('Spread')
        elif test_name == 'random':
            names.append('Random')
        else:
            names.append(test_name)
            
        clear = False
        
        test_res = pd.read_csv('Data/' + test_name + '.csv')

        sequence_len = test_res.shape[0]

        latencies = np.zeros((sequence_len,))
        throughputs = np.zeros((sequence_len,))
        counts = np.zeros((sequence_len,))
        tp_per_nodes = np.zeros((sequence_len,))
    
        l = test_res[package_name + '-avg_ms']
        t = test_res[package_name + '-completed']
        c = test_res[package_name + '-count']
        
        user_count_sequence = test_res[package_name + '-user_count']
        
        latencies = l
        throughputs = t
        counts = c
        tp_per_node = np.divide(t, c)
        
        tp_per_node = np.divide(t, c)
        tp_per_nodes = np.add(tp_per_nodes, tp_per_node)
        
        plt.figure(1+i, clear=clear)
        plt.plot(test_res[package_name + '-avg_ms'], marker=test_marker)
        plt.ylabel('Response Time (ms)')
        plt.xlabel('Time in Minutes')
        

        plt.figure(2+i, clear=clear)
        plt.plot(test_res[package_name + '-completed'], marker=test_marker)
        plt.ylabel('Throughput (Req/min)')
        plt.xlabel('Time in Minutes')

        plt.figure(3+i, clear=clear)
        plt.plot(test_res[package_name + '-count'], marker=test_marker)
        plt.ylabel('Container Count (#)')
        plt.xlabel('Time in Minutes')
        
        plt.figure(4+i)
        plt.plot(tp_per_nodes, marker=test_marker)
        plt.ylabel("Throughput / container")
        plt.xlabel('Time in Minutes')
        
        print("Image Name: ", name)
        print("Test Name: ", test_name)
        print('Response Time: {:4.02f} ({:4.02f})'.format(np.mean(latencies), np.std(latencies)))
        print('Thoughput: {:4.02f} ({:4.02f})'.format(np.mean(throughputs), np.std(throughputs)))
        print('Thoughput Per Node: {:4.02f} ({:4.02f})'.format(np.mean(tp_per_nodes), np.std(tp_per_nodes)))
        print('Total Package Count: {:4.02f} ({:4.02f})'.format(np.mean(counts), np.std(counts)))
        print('')
        
    plt.figure(1+i)
    shared_plot_stuff()
    
    rt_thresh = 0
    if package_name == 'nimamahmoudi/cpu-automated':
        rt_thresh = 600
    elif package_name == 'nimamahmoudi/fileio-automated':
        rt_thresh = 500
    elif package_name == 'nimamahmoudi/oltp-automated':
        rt_thresh = 600
    xlim,ylim = plt.xlim(), plt.ylim()
    plt.plot([xlim[0], xlim[1]], [rt_thresh, rt_thresh],
              linestyle="--")
    plt.legend(names + ['Threshold'])
    
    if save:
        plt.savefig("Images/ResponseTime_" + name + '_' + package_selection_method + ".png")
    plt.title(package_name.split('/')[-1])
    plt.figure(2+i)
    shared_plot_stuff()
    plt.legend(names)
    if save:
        plt.savefig("Images/Thoughput_" + name + '_' + package_selection_method + ".png")
    plt.title(package_name.split('/')[-1])
    plt.figure(3+i)
    shared_plot_stuff()
    plt.legend(names)
    if save:
        plt.savefig("Images/ContainerCount_" + name + '_' + package_selection_method + ".png")
    plt.title(package_name.split('/')[-1])
    plt.figure(4+i)
    shared_plot_stuff()
    plt.legend(names)
    if save:
        plt.savefig("Images/TP_Per_Container_" + name + '_' + package_selection_method + ".png")
    plt.title(package_name.split('/')[-1])
        
    i += 4